In [42]:
import QuantLib as ql
from math import pow, sqrt
import numpy as np
from scipy.optimize import root
import os
import pandas as pd
import functools


In [169]:
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()
calculation_date = ql.Date(8, 1, 2018)

spot = 273.97
ql.Settings.instance().evaluationDate = calculation_date

risk_free_rate = 0.05
dividend_rate = 0.0
yield_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, day_count))
dividend_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_rate, day_count))

In [170]:
def read_data(date,index="SPY",indicator="impliedV"):
    path = os.path.join("data",indicator,index,date+"_"+index+"~market__"+indicator+".csv")
    df = pd.read_csv(path)
    return(df)

In [171]:
df_iv = read_data("20180108")
df_iv["expiry"] = df_iv["expiry"].astype("str")
#df_iv["expiry"] = pd.to_datetime(df_iv["expiry"],format="%Y-%m-%d")

In [511]:
list_expiries = sorted(df_iv["expiry"].unique())[4:]

In [512]:
list_expiries

['20180126',
 '20180202',
 '20180216',
 '20180316',
 '20180420',
 '20180615',
 '20180720',
 '20180921',
 '20181221']

In [513]:
list_strikes = [df_iv[df_iv["expiry"]==exp]["strike"].values.tolist() for exp in list_expiries]

In [514]:
intersection_strikes_list = functools.reduce(lambda a,b : list(set(a) & set(b)),list_strikes)

In [495]:
df_iv_filtered = df_iv[(df_iv["expiry"].isin(list_expiries)) & (df_iv["strike"].isin(intersection_strikes_list))]

In [496]:
df_iv_filtered_pivot = df_iv_filtered.pivot(index="expiry",columns="strike",values="midImpliedV")

In [497]:
df_iv_filtered_pivot

strike,185.0,190.0,195.0,200.0,205.0,210.0,215.0,220.0,225.0,230.0,...,277.0,278.0,279.0,280.0,285.0,290.0,295.0,300.0,305.0,310.0
expiry,,,,,,,,,,,,,,,,,,,,,
20180420,0.309701,0.297456,0.284133,0.273452,0.259425,0.248814,0.237130,0.226235,0.214872,0.203307,...,0.089141,0.087186,0.085313,0.083504,0.077595,0.076162,0.078293,0.084708,0.089068,0.093555
20180615,0.279123,0.271159,0.261645,0.252980,0.243816,0.234690,0.225516,0.215631,0.206500,0.196725,...,0.100243,0.098337,0.096601,0.094872,0.087177,0.082121,0.080506,0.081582,0.084616,0.088552
20180720,0.270857,0.262481,0.254204,0.245798,0.237498,0.228980,0.220449,0.211831,0.202937,0.194053,...,0.107109,0.105224,0.103641,0.101918,0.094306,0.088106,0.084230,0.083227,0.084098,0.087172
20180921,0.259565,0.251693,0.244562,0.237450,0.229866,0.222303,0.214764,0.207354,0.199727,0.191983,...,0.115594,0.114130,0.112466,0.110938,0.103429,0.097122,0.091715,0.088165,0.086469,0.086845
20181221,0.245600,0.239659,0.233462,0.227082,0.221022,0.214573,0.208275,0.201982,0.195585,0.189119,...,0.125604,0.124341,0.122862,0.121360,0.115041,0.109227,0.103738,0.099243,0.095465,0.092823


# INPUTS :

In [498]:
expiration_dates = [ql.DateParser.parseFormatted(exp,'%Y%m%d') for exp in df_iv_filtered_pivot.index.tolist()]
strikes = df_iv_filtered_pivot.columns.tolist()
data = df_iv_filtered_pivot.values.tolist()

In [516]:
expiration_dates

[Date(20,4,2018),
 Date(15,6,2018),
 Date(20,7,2018),
 Date(21,9,2018),
 Date(21,12,2018)]

# HESTON CALIBRATION ON DATA

In [499]:
def setup_helpers(engine, expiration_dates, strikes, 
                  data, ref_date, spot, yield_ts, 
                  dividend_ts):
    heston_helpers = []
    grid_data = []
    for i, date in enumerate(expiration_dates):
        for j, s in enumerate(strikes):
            t = (date - ref_date )
            p = ql.Period(t, ql.Days)
            vols = data[i][j]
            helper = ql.HestonModelHelper(
                p, calendar, spot, s, 
                ql.QuoteHandle(ql.SimpleQuote(vols)),
                yield_ts, dividend_ts)
            helper.setPricingEngine(engine)
            heston_helpers.append(helper)
            grid_data.append((date, s))
    return heston_helpers, grid_data

def cost_function_generator(model, helpers,norm=False):
    def cost_function(params):
        params_ = ql.Array(list(params))
        model.setParams(params_)
        error = [h.calibrationError() for h in helpers]
        if norm:
            return np.sqrt(np.sum(np.abs(error)))
        else:
            return error
    return cost_function

def calibration_report(helpers, grid_data, detailed=False):
    avg = 0.0
    if detailed:
        print("%15s %25s %15s %15s %20s" % (
            "Strikes", "Expiry", "Market Value", 
             "Model Value", "Relative Error (%)"))
        print("="*100)
    for i, opt in enumerate(helpers):
        err = (opt.modelValue()/opt.marketValue() - 1.0)
        date,strike = grid_data[i]
        if detailed:
            print("%15.2f %25s %14.5f %15.5f %20.7f " % (
                strike, str(date), opt.marketValue(), 
                opt.modelValue(), 
                100.0*(opt.modelValue()/opt.marketValue() - 1.0)))
        avg += abs(err)
    avg = avg*100.0/len(helpers)
    if detailed: print("-"*100)
    summary = "Average Abs Error (%%) : %5.9f" % (avg)
    print(summary)
    return avg
    
def setup_model(_yield_ts, _dividend_ts, _spot, 
                init_condition=(0.02,0.2,0.5,0.1,0.01)):
    theta, kappa, sigma, rho, v0 = init_condition
    process = ql.HestonProcess(_yield_ts, _dividend_ts, 
                           ql.QuoteHandle(ql.SimpleQuote(_spot)), 
                           v0, kappa, theta, sigma, rho)
    model = ql.HestonModel(process)
    engine = ql.AnalyticHestonEngine(model) 
    return model, engine
summary= []

# OPTIMISATION AND RESULTS 

In [500]:
# clibration Using QuantLib Levenberg-Marquardt Solver

In [501]:
model1, engine1 = setup_model(
    yield_ts, dividend_ts, spot, 
    init_condition=(0.02,0.2,0.5,0.1,0.01))
heston_helpers1, grid_data1 = setup_helpers(
    engine1, expiration_dates, strikes, data, 
    calculation_date, spot, yield_ts, dividend_ts
)
initial_condition = list(model1.params())

In [502]:
%%time
lm = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
model1.calibrate(heston_helpers1, lm, 
                 ql.EndCriteria(500, 300, 1.0e-8,1.0e-8, 1.0e-8))
theta, kappa, sigma, rho, v0 = model1.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % \
    (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers1, grid_data1)
summary.append(["QL LM1", error] + list(model1.params()))

theta = 0.029416, kappa = 3.391330, sigma = 1.752326, rho = -0.623504, v0 = 0.004728
Average Abs Error (%) : 1.771193893
CPU times: user 1.25 s, sys: 11.3 ms, total: 1.26 s
Wall time: 1.27 s


In [503]:
## LEAST SQUARED ERROR OPTIMISATION

In [504]:
from scipy.optimize import least_squares
model3, engine3 = setup_model(
    yield_ts, dividend_ts, spot, 
    init_condition=(0.2,0.8,0.5,-0.5,0.02))
heston_helpers3, grid_data3 = setup_helpers(
    engine3, expiration_dates, strikes, data,
    calculation_date, spot, yield_ts, dividend_ts
)
initial_condition = list(model3.params())

In [505]:
%%time
cost_function = cost_function_generator(model3, heston_helpers3)
sol = least_squares(cost_function, initial_condition)
theta, kappa, sigma, rho, v0 = model3.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % \
    (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers3, grid_data3)
summary.append(["Scipy LS1", error] + list(model3.params()))

theta = 0.029417, kappa = 3.391130, sigma = 1.752264, rho = -0.623505, v0 = 0.004729
Average Abs Error (%) : 1.771167817
CPU times: user 2.36 s, sys: 11.4 ms, total: 2.37 s
Wall time: 2.38 s


In [519]:
sqrt(0.02)

0.1414213562373095

In [521]:
sqrt(0.004)

0.06324555320336758

In [508]:
# Diferential evolution

In [509]:
from scipy.optimize import differential_evolution
model4, engine4 = setup_model(yield_ts, dividend_ts, spot)
heston_helpers4, grid_data4 = setup_helpers(
    engine4, expiration_dates, strikes, data,
    calculation_date, spot, yield_ts, dividend_ts
)
initial_condition = list(model4.params())
bounds = [(0,1),(0.01,15), (0.01,1.), (-1,1), (0,1.0) ]


In [510]:
%%time
cost_function = cost_function_generator(
    model4, heston_helpers4, norm=True)
sol = differential_evolution(cost_function, bounds, maxiter=100)
theta, kappa, sigma, rho, v0 = model4.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % \
    (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers4, grid_data4)
summary.append(["Scipy DE1", error] + list(model4.params()))

theta = 0.044068, kappa = 0.996814, sigma = 1.000000, rho = -0.633347, v0 = 0.009492
Average Abs Error (%) : 2.709417779
CPU times: user 1min 25s, sys: 195 ms, total: 1min 26s
Wall time: 1min 26s


In [518]:
sqrt(0.009)

0.09486832980505137

In [403]:
#Basin Hopping Algorithm

In [506]:
from scipy.optimize import basinhopping
class MyBounds(object):
    def __init__(self, xmin=[0.,0.01,0.01,-1,0], xmax=[1,15,1,1,1.0] ):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin
bounds = [(0,1),(0.01,15), (0.01,1.), (-1,1), (0,1.0) ]
model5, engine5 = setup_model(
    yield_ts, dividend_ts, spot,
    init_condition=(0.02,0.2,0.5,0.1,0.01))
heston_helpers5, grid_data5 = setup_helpers(
    engine5, expiration_dates, strikes, data,
    calculation_date, spot, yield_ts, dividend_ts
)
initial_condition = list(model5.params())

In [507]:
%%time
mybound = MyBounds()
minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds }
cost_function = cost_function_generator(
    model5, heston_helpers5, norm=True)
sol = basinhopping(cost_function, initial_condition, niter=5,
                   minimizer_kwargs=minimizer_kwargs,
                   stepsize=0.005,
                   accept_test=mybound,
                   interval=10)
theta, kappa, sigma, rho, v0 = model5.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % \
    (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers5, grid_data5)
summary.append(["Scipy BH1", error] + list(model5.params()))

theta = 0.044023, kappa = 0.997852, sigma = 1.000000, rho = -0.633319, v0 = 0.009498
Average Abs Error (%) : 2.709397488
CPU times: user 1min 20s, sys: 659 ms, total: 1min 21s
Wall time: 1min 23s
